### Chatbot

In [1]:
import random 

import joblib

import sys
import os


import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from tensorflow.keras.models import Model

import scipy

In [2]:
df = pd.read_excel(r"database_intents.xlsx")

In [3]:
# Usar get_dummies en vez de las 3 funciones
df_oh = pd.concat([df, pd.get_dummies(df['Intent type'])], axis = 1)

In [4]:
df_train, df_test = train_test_split(df_oh, train_size = 0.7, test_size = 0.3, random_state = 42,
                                    shuffle = True, stratify = df_oh['Intent type'])

In [5]:
stopwords = []

In [6]:
cv = TfidfVectorizer(
    stop_words= stopwords,
    ngram_range=(1, 4),
    strip_accents='ascii',
    max_df=0.99,
    min_df=0,
    max_features=100
    )
cv.fit(df_train["Sentence"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.99, max_features=100, min_df=0,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=[], strip_accents='ascii', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
# import pickle

# with open('vectorizer.pkl', 'wb') as file:
#     pickle.dump(cv, file)

In [7]:
def processing(df, pretreatment = False, Tfidf = True, cv = None, stopwords = []):
  # Normalizamos y limpiamos el corpus 
  if pretreatment == True:
    df["Sentence"] = df['Sentence'].apply(lambda x: word_treatment(x))
    print("El corpus ha sido pretratado")
    
     # Transformamos nuestro corpus a un vector Tfidf
  if Tfidf == True:

    if cv == None:
      cv = TfidfVectorizer(
        stop_words= stopwords,
        ngram_range=(1, 4),
        strip_accents='ascii',
        max_df=0.99,
        min_df=0,
        max_features=100
      )
      cv.fit(df["Sentence"])
      X = cv.transform(df["Sentence"])
      print("Se ha realizado una vectorización Tfidf")
      return df, X, cv

    else:
      X = cv.transform(df["Sentence"])
      #print("Se ha realizado una vectorización Tfidf basado en el corpus suministrado por cv")
    return df, X
  else:
    return df


In [8]:
df_train, X_train, cv = processing(df_train)
df_test, X_test = processing(df_test, cv = cv)

Se ha realizado una vectorización Tfidf


In [9]:
y_train = df_train[["Greeting","Search","Suggestions"]]
y_test = df_test[["Greeting","Search","Suggestions"]]

In [10]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.7, 
test_size=0.3, random_state=42, shuffle=True, stratify=df_train["Intent type"])

In [11]:
shape= X_train.shape[1]

X_train.sort_indices()
X_validation.sort_indices()

In [12]:
def mlp_greeting(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [13]:
mlp_greeting = mlp_greeting(shape)
history = mlp_greeting.fit(X_train, np.asarray(y_train["Greeting"]).reshape(-1,1),
                  validation_data = (X_validation, np.asarray(y_validation["Greeting"]).reshape(-1,1)),
    epochs = 40,
    workers = 2, use_multiprocessing = True, verbose = 2)

Epoch 1/40


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.6902 - accuracy: 0.5517 - val_loss: 0.6685 - val_accuracy: 0.7600
Epoch 2/40
2/2 - 0s - loss: 0.6643 - accuracy: 0.7414 - val_loss: 0.6466 - val_accuracy: 0.7600
Epoch 3/40
2/2 - 0s - loss: 0.6449 - accuracy: 0.7414 - val_loss: 0.6266 - val_accuracy: 0.7600
Epoch 4/40
2/2 - 0s - loss: 0.6264 - accuracy: 0.7414 - val_loss: 0.6057 - val_accuracy: 0.7600
Epoch 5/40
2/2 - 0s - loss: 0.6010 - accuracy: 0.7759 - val_loss: 0.5828 - val_accuracy: 0.8000
Epoch 6/40
2/2 - 0s - loss: 0.5864 - accuracy: 0.8103 - val_loss: 0.5583 - val_accuracy: 0.8000
Epoch 7/40
2/2 - 0s - loss: 0.4982 - accuracy: 0.8966 - val_loss: 0.5284 - val_accuracy: 0.8000
Epoch 8/40
2/2 - 0s - loss: 0.5516 - accuracy: 0.7759 - val_loss: 0.5126 - val_accuracy: 0.8000
Epoch 9/40
2/2 - 0s - loss: 0.4489 - accuracy: 0.8621 - val_loss: 0.5060 - val_accuracy: 0.8000
Epoch 10/40
2/2 - 0s - loss: 0.4514 - accuracy: 0.8448 - val_loss: 0.5091 - val_accuracy: 0.8000
Epoch 11/40
2/2 - 0s - loss: 0.4291 - accuracy: 0.

In [14]:
def mlp_search(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [15]:
mlp_search = mlp_search(shape)
history = mlp_search.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs = 100,
    workers = 2, use_multiprocessing= True, verbose = 2)

Epoch 1/100


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense_5/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense_5/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense_5/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.6934 - accuracy: 0.7586 - val_loss: 0.5841 - val_accuracy: 0.7600
Epoch 2/100
2/2 - 0s - loss: 0.5874 - accuracy: 0.7586 - val_loss: 0.5228 - val_accuracy: 0.7600
Epoch 3/100
2/2 - 0s - loss: 0.4649 - accuracy: 0.7586 - val_loss: 0.4847 - val_accuracy: 0.7600
Epoch 4/100
2/2 - 0s - loss: 0.4855 - accuracy: 0.7586 - val_loss: 0.4483 - val_accuracy: 0.7600
Epoch 5/100
2/2 - 0s - loss: 0.4101 - accuracy: 0.7586 - val_loss: 0.4208 - val_accuracy: 0.7600
Epoch 6/100
2/2 - 0s - loss: 0.4046 - accuracy: 0.7586 - val_loss: 0.3860 - val_accuracy: 0.7600
Epoch 7/100
2/2 - 0s - loss: 0.3580 - accuracy: 0.7586 - val_loss: 0.3579 - val_accuracy: 0.7600
Epoch 8/100
2/2 - 0s - loss: 0.3240 - accuracy: 0.7586 - val_loss: 0.3333 - val_accuracy: 0.7600
Epoch 9/100
2/2 - 0s - loss: 0.2915 - accuracy: 0.7586 - val_loss: 0.3125 - val_accuracy: 0.7600
Epoch 10/100
2/2 - 0s - loss: 0.3083 - accuracy: 0.7586 - val_loss: 0.2948 - val_accuracy: 0.7600
Epoch 11/100
2/2 - 0s - loss: 0.2292 - ac

Epoch 85/100
2/2 - 0s - loss: 0.0643 - accuracy: 0.9828 - val_loss: 1.2344 - val_accuracy: 0.9600
Epoch 86/100
2/2 - 0s - loss: 0.0644 - accuracy: 0.9828 - val_loss: 1.2418 - val_accuracy: 0.9600
Epoch 87/100
2/2 - 0s - loss: 0.0224 - accuracy: 1.0000 - val_loss: 1.2476 - val_accuracy: 0.9600
Epoch 88/100
2/2 - 0s - loss: 0.0221 - accuracy: 1.0000 - val_loss: 1.2519 - val_accuracy: 0.9600
Epoch 89/100
2/2 - 0s - loss: 0.0640 - accuracy: 0.9828 - val_loss: 1.2553 - val_accuracy: 0.9600
Epoch 90/100
2/2 - 0s - loss: 0.0638 - accuracy: 0.9828 - val_loss: 1.2581 - val_accuracy: 0.9600
Epoch 91/100
2/2 - 0s - loss: 0.0209 - accuracy: 1.0000 - val_loss: 1.2604 - val_accuracy: 0.9600
Epoch 92/100
2/2 - 0s - loss: 0.0204 - accuracy: 1.0000 - val_loss: 1.2620 - val_accuracy: 0.9600
Epoch 93/100
2/2 - 0s - loss: 0.0199 - accuracy: 1.0000 - val_loss: 1.2631 - val_accuracy: 0.9600
Epoch 94/100
2/2 - 0s - loss: 0.0647 - accuracy: 0.9828 - val_loss: 1.2398 - val_accuracy: 0.9600
Epoch 95/100
2/2 - 0

In [16]:
def mlp_suggestion(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.01)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer = opt)
# return our model
    return model

In [17]:
mlp_suggestion = mlp_suggestion(shape)
history = mlp_suggestion.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs = 250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Epoch 1/250


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_2/dense_10/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_2/dense_10/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_2/dense_10/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.6850 - accuracy: 0.7586 - val_loss: 0.6709 - val_accuracy: 0.7600
Epoch 2/250
2/2 - 0s - loss: 0.6668 - accuracy: 0.7586 - val_loss: 0.6514 - val_accuracy: 0.7600
Epoch 3/250
2/2 - 0s - loss: 0.6403 - accuracy: 0.7586 - val_loss: 0.6310 - val_accuracy: 0.7600
Epoch 4/250
2/2 - 0s - loss: 0.6356 - accuracy: 0.7586 - val_loss: 0.6099 - val_accuracy: 0.7600
Epoch 5/250
2/2 - 0s - loss: 0.6085 - accuracy: 0.7586 - val_loss: 0.5907 - val_accuracy: 0.7600
Epoch 6/250
2/2 - 0s - loss: 0.5877 - accuracy: 0.7586 - val_loss: 0.5724 - val_accuracy: 0.7600
Epoch 7/250
2/2 - 0s - loss: 0.5586 - accuracy: 0.7586 - val_loss: 0.5536 - val_accuracy: 0.7600
Epoch 8/250
2/2 - 0s - loss: 0.5376 - accuracy: 0.7586 - val_loss: 0.5374 - val_accuracy: 0.7600
Epoch 9/250
2/2 - 0s - loss: 0.5518 - accuracy: 0.7586 - val_loss: 0.5231 - val_accuracy: 0.7600
Epoch 10/250
2/2 - 0s - loss: 0.5358 - accuracy: 0.7586 - val_loss: 0.5110 - val_accuracy: 0.7600
Epoch 11/250
2/2 - 0s - loss: 0.5120 - ac

Epoch 85/250
2/2 - 0s - loss: 0.1598 - accuracy: 0.9483 - val_loss: 0.1829 - val_accuracy: 0.9600
Epoch 86/250
2/2 - 0s - loss: 0.1558 - accuracy: 0.9655 - val_loss: 0.1888 - val_accuracy: 0.9600
Epoch 87/250
2/2 - 0s - loss: 0.1794 - accuracy: 0.9310 - val_loss: 0.1993 - val_accuracy: 0.9600
Epoch 88/250
2/2 - 0s - loss: 0.2122 - accuracy: 0.8966 - val_loss: 0.2102 - val_accuracy: 0.9600
Epoch 89/250
2/2 - 0s - loss: 0.1754 - accuracy: 0.9310 - val_loss: 0.2202 - val_accuracy: 0.9600
Epoch 90/250
2/2 - 0s - loss: 0.1736 - accuracy: 0.9310 - val_loss: 0.2295 - val_accuracy: 0.9600
Epoch 91/250
2/2 - 0s - loss: 0.1384 - accuracy: 0.9655 - val_loss: 0.2397 - val_accuracy: 0.9600
Epoch 92/250
2/2 - 0s - loss: 0.1385 - accuracy: 0.9655 - val_loss: 0.2477 - val_accuracy: 0.9600
Epoch 93/250
2/2 - 0s - loss: 0.1931 - accuracy: 0.9138 - val_loss: 0.2548 - val_accuracy: 0.9200
Epoch 94/250
2/2 - 0s - loss: 0.1762 - accuracy: 0.9310 - val_loss: 0.2632 - val_accuracy: 0.9200
Epoch 95/250
2/2 - 0

Epoch 168/250
2/2 - 0s - loss: 0.1122 - accuracy: 0.9655 - val_loss: 0.3630 - val_accuracy: 0.9600
Epoch 169/250
2/2 - 0s - loss: 0.1121 - accuracy: 0.9655 - val_loss: 0.3668 - val_accuracy: 0.9600
Epoch 170/250
2/2 - 0s - loss: 0.1383 - accuracy: 0.9483 - val_loss: 0.3701 - val_accuracy: 0.9600
Epoch 171/250
2/2 - 0s - loss: 0.1119 - accuracy: 0.9655 - val_loss: 0.3729 - val_accuracy: 0.9600
Epoch 172/250
2/2 - 0s - loss: 0.1383 - accuracy: 0.9483 - val_loss: 0.3747 - val_accuracy: 0.9600
Epoch 173/250
2/2 - 0s - loss: 0.1436 - accuracy: 0.9483 - val_loss: 0.3788 - val_accuracy: 0.9600
Epoch 174/250
2/2 - 0s - loss: 0.1379 - accuracy: 0.9483 - val_loss: 0.3822 - val_accuracy: 0.9600
Epoch 175/250
2/2 - 0s - loss: 0.0847 - accuracy: 0.9828 - val_loss: 0.3849 - val_accuracy: 0.9600
Epoch 176/250
2/2 - 0s - loss: 0.0846 - accuracy: 0.9828 - val_loss: 0.3872 - val_accuracy: 0.9600
Epoch 177/250
2/2 - 0s - loss: 0.0590 - accuracy: 1.0000 - val_loss: 0.3903 - val_accuracy: 0.9600
Epoch 178/

In [18]:
def mlp_farewell(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate = 0.05)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [19]:
mlp_farewell = mlp_farewell(shape)
history = mlp_farewell.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs = 50,
    workers = 2, use_multiprocessing= True, verbose = 2)

Epoch 1/50


C:\Users\Usuario\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_3/dense_15/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_3/dense_15/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_3/dense_15/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


2/2 - 1s - loss: 0.6783 - accuracy: 0.6379 - val_loss: 0.6283 - val_accuracy: 0.7600
Epoch 2/50
2/2 - 0s - loss: 0.5891 - accuracy: 0.7586 - val_loss: 0.5562 - val_accuracy: 0.7600
Epoch 3/50
2/2 - 0s - loss: 0.5265 - accuracy: 0.7586 - val_loss: 0.5288 - val_accuracy: 0.7600
Epoch 4/50
2/2 - 0s - loss: 0.5250 - accuracy: 0.7586 - val_loss: 0.4643 - val_accuracy: 0.7600
Epoch 5/50
2/2 - 0s - loss: 0.4145 - accuracy: 0.7586 - val_loss: 0.4098 - val_accuracy: 0.7600
Epoch 6/50
2/2 - 0s - loss: 0.3965 - accuracy: 0.7586 - val_loss: 0.3699 - val_accuracy: 0.7600
Epoch 7/50
2/2 - 0s - loss: 0.3492 - accuracy: 0.7586 - val_loss: 0.3423 - val_accuracy: 0.7600
Epoch 8/50
2/2 - 0s - loss: 0.4039 - accuracy: 0.7586 - val_loss: 0.3123 - val_accuracy: 0.7600
Epoch 9/50
2/2 - 0s - loss: 0.3799 - accuracy: 0.7586 - val_loss: 0.2874 - val_accuracy: 0.7600
Epoch 10/50
2/2 - 0s - loss: 0.3862 - accuracy: 0.7586 - val_loss: 0.2725 - val_accuracy: 0.7600
Epoch 11/50
2/2 - 0s - loss: 0.3241 - accuracy: 0.

In [21]:
mlp_greeting.save('model_greetings.h5')
mlp_search.save('model_search.h5')
mlp_suggestion.save('model_suggestion.h5')
mlp_farewell.save('model_farewell.h5')

# CHATBOT TEST

In [95]:
mlp_greeting = keras.models.load_model('model_greetings.h5')
mlp_search = keras.models.load_model('model_search.h5')
mlp_suggestion = keras.models.load_model('model_suggestion.h5')
mlp_farewell = keras.models.load_model('model_farewell.h5')
# cv = joblib.load("vectorizer.pkl")

In [22]:
def chatbot():
    
    input_text = input()
    
    test = pd.DataFrame(data = {'Sentence': [input_text]})
    df_test_proc, test_proc = processing(test, cv = cv)
    scipy.sparse.csr_matrix.sort_indices(test_proc)
    
    gret_prob = mlp_greeting.predict(test_proc)
    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    farewell_prob = mlp_farewell.predict(test_proc)
    
    probs = [gret_prob, search_prob, sugg_prob, farewell_prob]
    print(probs)
    idx = np.argmax(probs)
    
    if idx == 0:
        print("Esto es un saludo")
    elif idx == 1:
        print("Esto es una búsqueda")
    elif idx == 2:
        print("Esto es una sugerencia")
    else:
        print("Esto es una despedida")
        
#     print('¿Hemos acertado?')
    
#     respuesta = input()
#     if (respuesta == 'No' or respuesta == 'no'):
#         probs = np.delete(probs, idx)
#         idx_2 = np.argmax(probs)
        
#         if idx == 0:
#             if idx_2 == 0:
#                 print("Esto es una búsqueda")
#             else:
#                 print("Esto es una sugerencia")
#         elif idx == 1:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una sugerencia")
#         else:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una búsqueda")
#     else:
#         print('¡Genial! ¡Hemos acertado!')
    

In [23]:
chatbot()

Hi
[array([[1.]], dtype=float32), array([[0.]], dtype=float32), array([[8.298176e-22]], dtype=float32), array([[0.]], dtype=float32)]
Esto es un saludo


In [24]:
chatbot()

Hello
[array([[0.113911]], dtype=float32), array([[2.7741018e-18]], dtype=float32), array([[6.400263e-10]], dtype=float32), array([[3.3010424e-12]], dtype=float32)]
Esto es un saludo


In [27]:
chatbot()

I want to know more about Paris
[array([[0.113911]], dtype=float32), array([[8.6640784e-26]], dtype=float32), array([[6.7579206e-11]], dtype=float32), array([[9.015104e-07]], dtype=float32)]
Esto es un saludo


In [28]:
chatbot()

Search for Barcelona
[array([[0.113911]], dtype=float32), array([[0.9307531]], dtype=float32), array([[0.25079298]], dtype=float32), array([[0.8679251]], dtype=float32)]
Esto es una búsqueda


* Correctamente
 * La mayoría de saludos
 * I want to know about Data Science (búsqueda)
 * I want to know things connected to la Sagrada Familia (sugerencia)
 * Watcha doing'? (saludo)
 * Tell me things related to Data Science (sugerencia)

* Incorrectamente
 * What is Data Science (sugerencia)
 * Tell me things related to Napoleon (saludo)
 * Who is Cristobal Colon (saludo)



In [ ]:
chatbot()

* Opciones
 * Mostrar un comando *!options* que te de las opciones para hacer.
 * Detectar un intent
 * Mostrarlo al inicio


* Headers
 * Dar las opciones una vez se detecta intent de búsqueda búsqueda
   * *What is data science?* Y mostrarle los heades para qué elija qué queire saber 
 * Intent nuevo de header

In [ ]:
chatbot()

In [ ]:
def Intent_detection_function(keyboard):
    gret_prob = mlp_greeting.predict_proba(test_proc)
    search_prob = mlp_search.predict_proba(test_proc)
    sugg_prob = mlp_suggestion.predict_proba(test_proc)
    farewell_prob = mlp_farewell.predict_proba(test_proc)
    cv = joblib.load("vectorizer.pkl")


    input_text = pd.DataFrame(data={'Sentence': [keyboard]})
    _, test_proc = cv.transform(input_text['Sentence'])
    #df_test_proc, test_proc = processing(test, cv=cv)

    test_proc = test_proc.toarray()

    gret_prob = mlp_greeting.predict(test_proc)

    search_prob = mlp_search.predict(test_proc)
    sugg_prob = mlp_suggestion.predict(test_proc)
    fare_prob = mlp_farewell.predict(test_proc)


    probs = [gret_prob, search_prob, sugg_prob, fare_prob]
    print(probs)
    idx = np.argmax(probs)

    if idx == 0:
        intent = "Greeting"
        keyword = None
    elif idx == 1:
        intent = "Search"
        keyword = getEntities(keyboard)
    elif idx == 2:
        intent = "Suggestions"
        keyword = getEntities(keyboard)
    elif idx == 3:
        intent = "Farewell"
        keyword = None

    return intent, keyword

In [ ]:
Intent_detection_function()